In [6]:
import psycopg2 as pg2
import pandas as pd
import numpy as np
import pickle

In [5]:
%%html
<style>
table {float:left}
</style>

In [3]:
with open('data/cs_df_M.pkl', 'rb') as f:
    df = pickle.load(f)

In [128]:
df.head(20)

,item_name,date_str,median_sell_price,quantity,est_release_str,days_since_release,gun_type,skin,quality,gun_bool,StatTrak,souvenir,keys,capsules,stickers,date,est_release,timestamp
0,Glock-18 | Weasel (Battle-Scarred),19 Aug 2016,6.193,135,19 Aug 2016,0,Glock-18,Weasel,Battle-Scarred,True,False,False,False,False,False,1.471583e+09,1.471583e+09,2016-08-19
1,Glock-18 | Weasel (Battle-Scarred),20 Aug 2016,5.103,166,19 Aug 2016,1,Glock-18,Weasel,Battle-Scarred,True,False,False,False,False,False,1.471669e+09,1.471583e+09,2016-08-20
2,Glock-18 | Weasel (Battle-Scarred),21 Aug 2016,4.721,142,19 Aug 2016,2,Glock-18,Weasel,Battle-Scarred,True,False,False,False,False,False,1.471756e+09,1.471583e+09,2016-08-21
3,Glock-18 | Weasel (Battle-Scarred),22 Aug 2016,4.302,98,19 Aug 2016,3,Glock-18,Weasel,Battle-Scarred,True,False,False,False,False,False,1.471842e+09,1.471583e+09,2016-08-22
4,Glock-18 | Weasel (Battle-Scarred),23 Aug 2016,3.898,125,19 Aug 2016,4,Glock-18,Weasel,Battle-Scarred,True,False,False,False,False,False,1.471928e+09,1.471583e+09,2016-08-23
5,Glock-18 | Weasel (Battle-Scarred),24 Aug 2016,3.619,131,19 Aug 2016,5,Glock-18,Weasel,Battle-Scarred,True,False,False,False,False,False,1.472015e+09,1.471583e+09,2016-08-24
6,Glock-18 | Weasel (Battle-Scarred),25 Aug 2016,3.323,122,19 Aug 2016,6,Glock-18,Weasel,Battle-Scarred,True,False,False,False,False,False,1.472101e+09,1.471583e+09,2016-08-25
7,Glock-18 | Weasel (Battle-Scarred),26 Aug 2016,3.384,139,19 Aug 2016,7,Glock-18,Weasel,Battle-Scarred,True,False,False,False,False,False,1.472188e+09,1.471583e+09,2016-08-26
8,Glock-18 | Weasel (Battle-Scarred),27 Aug 2016,3.422,128,19 Aug 2016,8,Glock-18,Weasel,Battle-Scarred,True,False,False,False,False,False,1.472274e+09,1.471583e+09,2016-08-27
9,Glock-18 | Weasel (Battle-Scarred),28 Aug 2016,3.329,151,19 Aug 2016,9,Glock-18,Weasel,Battle-Scarred,True,False,False,False,False,False,1.472360e+09,1.471583e+09,2016-08-28


## SQL Table Schemas:

#### ID Table:

item_id | item_name
---|---
int|str

#### Sales Table:

Could add days_since_release here or find a way to easily calculate it from a release date

item_id |date | price | quant | (days since release?)
---|---|---|---|---
int|float (unix)|float|int|(int?)

#### Details/Masking Table:

item_id | is_gun | is_capsule | is_key | is_sticker | is_case
---|---|---|---|---|---
int|bool|bool|bool|bool|bool

#### Guns Table:

item_id | gun_type | skin_name | quality | StatTrak | Souvenir
--|---|---|---|---|---
int|str|str|str|bool|bool

#### Release Date Table? Add to ID table? 

item_id | release_date
---|---
int|float (unix)

---

### Starting with the ID table and the sales table

In [142]:
def connect():
    return pg2.connect(dbname='steam_capstone', host='localhost'), pg2.connect(dbname='steam_capstone', host='localhost').cursor()

In [7]:
# This created the table, doesn't need to be run again

# conn = pg2.connect(dbname='postgres', host='localhost')
# conn.autocommit = True
# cur = conn.cursor()
# cur.execute('CREATE DATABASE steam_capstone;')
# conn.close()

#### Finding longest item name for `item_name` datatype (ended up using `text`)

In [12]:
max(map(len, df.item_name))

65

In [143]:
conn, cur = connect()

In [117]:
query = '''
        CREATE TABLE id (
            item_id SERIAL PRIMARY KEY, 
            item_name text
        );
        '''
cur.execute(query)

In [118]:
query = '''
        CREATE TABLE sales (
            item_id INTEGER REFERENCES id(item_id) NOT NULL, 
            date date,
            price real,
            quantity integer
        );
        '''
cur.execute(query)

In [119]:
conn.commit()

In [120]:
cur.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
for table in cur.fetchall():
    print(table)

('id',)
('sales',)


In [121]:
conn.close()

### Inserting the data

In [122]:
def insert_item_names(names):
    sql = "INSERT INTO id(item_name) VALUES(%s)"
    conn = None
    try:
        # connect to the PostgreSQL database
        conn = pg2.connect(dbname='steam_capstone', host='localhost')
        # create a new cursor
        cur = conn.cursor()
        # execute the INSERT statement
        cur.executemany(sql, names)
        # commit the changes to the database
        conn.commit()
        # close communication with the database
        cur.close()
    except (Exception, pg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [123]:
names = list(map(lambda x: (x,), df.item_name.unique()))

In [124]:
insert_item_names(names)

In [126]:
conn, cur = connect()

In [127]:
cur.execute('SELECT * FROM id LIMIT 5;')
cur.fetchall()

[(1, 'Glock-18 | Weasel (Battle-Scarred)'),
 (2, 'PP-Bizon | Photic Zone (Minimal Wear)'),
 (3, 'SSG 08 | Ghost Crusader (Field-Tested)'),
 (4, 'Spectrum Case Key'),
 (5, 'Sticker | shroud (Foil) | Krakow 2017')]

In [101]:
conn.close()

#### Insert sales data

In [129]:
sales_records = list(map(tuple, df[['item_name', 'date_str', 'median_sell_price', 'quantity']].values))

In [131]:
# def insert_sales(sales):
#     sql = """INSERT INTO sales(item_id, date, price, quantity) 
#              VALUES((SELECT item_id FROM id WHERE item_name=%s), to_date(%s, 'DD Mon YYYY'), %s, %s);"""
#     conn = None
#     try:
#         # connect to the PostgreSQL database
#         conn = pg2.connect(dbname='steam_capstone', host='localhost')
#         # create a new cursor
#         cur = conn.cursor()
#         # execute the INSERT statement
#         cur.executemany(sql, sales)
#         # commit the changes to the database
#         conn.commit()
#         # close communication with the database
#         cur.close()
#     except (Exception, pg2.DatabaseError) as error:
#         print(error)
#     finally:
#         if conn is not None:
#             conn.close()

In [ ]:
insert_sales(sales_records)

##### The above was too slow, going to remove the foreign key, add an item_name column, COPY the data, then generate the foreign key

In [168]:
conn = pg2.connect(dbname='steam_capstone', host='localhost')
cur = conn.cursor()
cur.execute('DROP TABLE sales;')
query = '''
        CREATE TABLE sales (
            item_id integer,
            item_name text,
            date date,
            price real,
            quantity integer
        );
        '''
cur.execute(query)
conn.commit()
conn.close()

Generating a dataframe with the desired columns

In [ ]:
df_sales = df[['item_name', 'date_str', 'median_sell_price', 'quantity']]
df_sales['item_id'] = [keys_dict[item] for item in df_sales.item_name]
df_sales = df_sales[['item_id', 'item_name', 'date_str', 'median_sell_price', 'quantity']]

Streaming a csv to the sales table

In [169]:
# from https://gist.github.com/ellisvalentiner/63b083180afe54f17f16843dd51f4394

from io import StringIO

# Get a database connection
conn = pg2.connect(dbname='steam_capstone', host='localhost')

# Initialize a string buffer
sio = StringIO()
sio.write(df_sales.to_csv(index=None, header=None, sep='\t'))  # Write the Pandas DataFrame as a csv to the buffer
sio.seek(0)  # Be sure to reset the position to the start of the stream

# Copy the string buffer to the database, as if it were an actual file
with conn.cursor() as c:
    c.copy_from(sio, "sales")
    conn.commit()
conn.close() # I think this is handled with the with statement above, but just incase

#### Below didn't work before

In [147]:
# Add foreign key
conn, cur = connect()
cur.execute("""
            UPDATE sales SET item_id = id.item_id
            FROM id WHERE sales.item_name = id.item_name;
            """)
cur.execute("""ALTER TABLE sales ADD CONSTRAINT id_item_id_seq FOREIGN KEY (item_id) REFERENCES id(item_id);""")
cur.execute("""
            ALTER TABLE sales 
            ALTER COLUMN item_id SET NOT NULL;
            """)
conn.commit()
conn.close()